In [1]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git@fix/lerobot_openpi

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
  Cloning https://github.com/huggingface/transformers.git (to revision fix/lerobot_openpi) to /tmp/pip-req-build-gv0u_nis
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gv0u_nis
  Running command git checkout -b fix/lerobot_openpi --track origin/fix/lerobot_openpi
  Switched to a new branch 'fix/lerobot_openpi'
  branch 'fix/lerobot_openpi' set up to track 'origin/fix/lerobot_openpi'.
  Resolved https://github.com/huggingface/transformers.git to commit dcddb970176382c0fcf4521b0c0e6fc15894dfe0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.3-py3-none-any.whl size=10827916 sha256=8e5526afe523346601589ee0b65b19db5cce41247cba7a1776d9c0f86517f

In [1]:
import random
import numpy as np
import os
import torch
import json
from PIL import Image
from src.env.env import RILAB_OMY_ENV
from torchvision import transforms

from lerobot.datasets.utils import dataset_to_policy_features
from lerobot.policies.pi0.modeling_pi0 import PI0Policy
from lerobot.processor import PolicyAction, PolicyProcessorPipeline
from lerobot.processor.converters import (
    batch_to_transition,
    policy_action_to_transition,
    transition_to_batch,
    transition_to_policy_action,
)
from lerobot.utils.constants import POLICY_POSTPROCESSOR_DEFAULT_NAME, POLICY_PREPROCESSOR_DEFAULT_NAME

import glfw

/home/rilab/ros2_ws/src/Lerobot-MujoCo-Custom-Policy-Tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [ ]:
'''
Meta data is for loading dataset statistics and feature information
'''
repo_id_or_path = 'Jeongeun/tutorial_v2_pi0' # Use this for loading pretrained model from the hub
device = 'cuda'

policy = PI0Policy.from_pretrained(repo_id_or_path)
_ = policy.to(device)

The PI0 model is a direct port of the OpenPI implementation. 
This implementation follows the original OpenPI structure for compatibility. 
Original implementation: https://github.com/Physical-Intelligence/openpi
Loading model from: Jeongeun/tutorial_v2_pi0


In [ ]:
'''
If you want to change number of actions in chunk to be executed plz change
policy.config.n_action_steps = YOUR_DESIRED_NUMBER
'''

In [ ]:
preprocessor = PolicyProcessorPipeline.from_pretrained(
                pretrained_model_name_or_path=repo_id_or_path,
                config_filename= f"{POLICY_PREPROCESSOR_DEFAULT_NAME}.json",
                overrides={},
                to_transition=batch_to_transition,
                to_output=transition_to_batch,
            )

postprocessor = PolicyProcessorPipeline.from_pretrained(
                pretrained_model_name_or_path=repo_id_or_path,
                config_filename= f"{POLICY_POSTPROCESSOR_DEFAULT_NAME}.json",
                overrides={},
                to_transition=policy_action_to_transition,
                to_output=transition_to_policy_action,
            )

In [4]:
'''
Load environment configuration and initialize environments
'''
# Evaluation Configuration
TEST_EPISODES = 20
MAX_EPISODE_STEPS = 10_000

In [5]:
batch = {
    'observation.state': np.zeros((1, 8), dtype=np.float32),
    'observation.image': np.zeros((1, 3, 256, 256), dtype=np.float32),
    'observation.wrist_image': np.zeros((1, 3, 256, 256), dtype=np.float32),
    'task': ['move the object to the target position']
}
batch = preprocessor(batch)  # to initialize the processors
_ = policy.select_action(batch)  # to initialize the model

## Load Environment

In [6]:
config_file_path = './configs/train.json'
with open(config_file_path) as f:
    env_conf = json.load(f)
omy_env = RILAB_OMY_ENV(cfg=env_conf, seed=0, 
                        action_type='joint', 
                        obs_type='joint_pos',
                        vis_mode = 'teleop')


-----------------------------------------------------------------------------
name:[tabletop_env] dt:[0.002] HZ:[500]
 n_qpos:[34] n_qvel:[31] n_qacc:[31] n_ctrl:[8]
 integrator:[IMPLICITFAST]

n_body:[25]
 [0/25] [world] mass:[0.00]kg
 [1/25] [front_object_table] mass:[1.00]kg
 [2/25] [camera] mass:[0.00]kg
 [3/25] [camera2] mass:[0.00]kg
 [4/25] [camera3] mass:[0.00]kg
 [5/25] [base] mass:[3.59]kg
 [6/25] [link1] mass:[2.06]kg
 [7/25] [link2] mass:[3.68]kg
 [8/25] [link3] mass:[2.39]kg
 [9/25] [link4] mass:[1.40]kg
 [10/25] [link5] mass:[1.40]kg
 [11/25] [link6] mass:[0.65]kg
 [12/25] [camera_center] mass:[0.00]kg
 [13/25] [tcp_link] mass:[0.32]kg
 [14/25] [rh_p12_rn_r1] mass:[0.07]kg
 [15/25] [rh_p12_rn_r2] mass:[0.02]kg
 [16/25] [rh_p12_rn_l1] mass:[0.07]kg
 [17/25] [rh_p12_rn_l2] mass:[0.02]kg
 [18/25] [body_obj_wooden_cabinet] mass:[2.24]kg
 [19/25] [body_obj_wooden_cabinet_top] mass:[3.00]kg
 [20/25] [body_obj_wooden_cabinet_middle] mass:[3.00]kg
 [21/25] [body_obj_wooden_cabin

In [7]:
def get_default_transform():
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL [0–255] -> FloatTensor [0.0–1.0], shape C×H×W
    ])
IMG_TRANSFORM = get_default_transform()

## Evaluate

In [8]:
'''
Run one evaluation episode
'''
def run_one_episode():
    omy_env.reset()
    policy.reset()
    observation = omy_env.get_observation()
    omy_env.env.tick = 0
    while omy_env.env.is_viewer_alive() and omy_env.env.tick < MAX_EPISODE_STEPS:
        omy_env.step_env()
        if omy_env.env.loop_every(HZ = 20):
            success = omy_env.check_success()
            if success: break
            if omy_env.env.is_key_pressed_once(glfw.KEY_Z):
                break  # for debugging: press 'z' to end the episode
            agent_image, wrist_image = omy_env.grab_image(return_side=False)
            # # resize to 256x256
            frame = {
                "observation.state": observation.astype(np.float32),
                'task': [env_conf['language_instruction']]
            }
            agent_image = Image.fromarray(agent_image)
            wrist_image = Image.fromarray(wrist_image)
            agent_image = agent_image.resize((256, 256))
            wrist_image = wrist_image.resize((256, 256))
            agent_image = IMG_TRANSFORM(agent_image)
            wrist_image = IMG_TRANSFORM(wrist_image)
            frame["observation.image"] = agent_image
            frame["observation.wrist_image"] = wrist_image
            # numpy to torch
            frame = {k: torch.tensor(v, dtype=torch.float32).unsqueeze(0) if isinstance(v, np.ndarray) else v for k, v in frame.items()}
            # pre-process the frame
            frame = preprocessor(frame)
            # select action
            action = policy.select_action(frame)
            # post-process the action
            action = postprocessor(action)
            action = action.squeeze(0).cpu().numpy()
            observation = omy_env.step(action, gripper_mode='continuous')
            omy_env.render()
    return success

In [9]:
'''
Run evaluation over multiple episodes
'''
results = []
for episode in range(TEST_EPISODES):
    success = run_one_episode()
    results.append(success)
    print(f"Episode {episode+1}/{TEST_EPISODES} - Success: {success}")
omy_env.env.close_viewer()
# log average success rate
avg_success = sum(results) / len(results)
print(f"Average Success Rate over {TEST_EPISODES} episodes: {avg_success*100:.2f}%")


['top', 'close']
DONE INITIALIZATION
Episode 1/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 2/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 3/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 4/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 5/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 6/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 7/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 8/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 9/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 10/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 11/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 12/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 13/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 14/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 15/20 - Success: True
[